In [5]:
import ujson

In [6]:
filename = 'data.json' ## insert filename here
with open(filename) as f:
    data = ujson.load(f)

TRAINING_DATA = []

In [7]:
for d in data:
    tuple_1 = d['data']['response']
    tuple_2 = {}
    entities = []
    results = d['annotations'][0]['result']
    for x in results:
        entities.append((x['value']['start'],  x['value']['end'], x['value']['labels'][0]))
    tuple_2['entities'] = entities
    TRAINING_DATA.append((tuple_1, tuple_2))

In [10]:
from spacy.tokens import DocBin
import spacy
db = DocBin()
nlp = spacy.blank("en")

bad = 0

for text, annotations in TRAINING_DATA:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations['entities']:
        # print(text, text[start:end], label)
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    try:
        doc.ents = ents
        db.add(doc)
    except:
        bad += 1
db.to_disk("./train.spacy")
print(bad)


7


In [13]:
!python -m spacy train config.cfg --paths.train train.spacy --paths.dev train.spacy --output output_folder

ℹ Saving to output directory: output_folder
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-01 00:00:30,489] [INFO] Set up nlp object from config
[2022-12-01 00:00:30,506] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-12-01 00:00:30,512] [INFO] Created vocabulary
[2022-12-01 00:00:30,514] [INFO] Finished initializing nlp object
[2022-12-01 00:00:30,765] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     69.37   18.15   14.17   25.24    0.18
 15     200        245.27   5570.43   99.23   99.16   99.30    0.99
 35     400        191.20    458.68   99.72   99.58   99.86    1.00
 59     600        150.94    180.51 

In [23]:
!python -m spacy package "/Users/chris/Documents/UR/SENIOR/CSC412/Projects/Final/CLP/output_folder/model-best" "/Users/chris/Documents/UR/SENIOR/CSC412/Projects/Final/CLP/CoffeeModel" --build wheel

ℹ Building package artifacts: wheel

✘ Can't load pipeline meta.json
/Users/chris/Documents/UR/SENIOR/CSC412/Projects/Final/CLP/output_folder/model-best/meta.json



In [20]:
from spacy_huggingface_hub import push

result = push("/Users/chris/Documents/UR/SENIOR/CSC412/Projects/Final/CLP/CoffeeModel/en_Coff_Ev1-1.0.4")


✘ Not a valid .whl file:
/Users/chris/Documents/UR/SENIOR/CSC412/Projects/Final/CLP/CoffeeModel/en_Coff_Ev1-1.0.4.
Make sure to run `spacy package` with `--build wheel` to generate a wheel for
you pipeline.



SystemExit: 1